In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [2]:
# !wget https://d37ci6vzurychx.cloudfront.net/trip-data/fhv_tripdata_2021-02.parquet

In [3]:
df = spark.read.parquet('fhv_tripdata_2021-02.parquet')

In [4]:
df.registerTempTable('fhvhv_data')

C:\Users\user\anaconda3\lib\site-packages\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [5]:
df.columns

['dispatching_base_num',
 'pickup_datetime',
 'dropOff_datetime',
 'PUlocationID',
 'DOlocationID',
 'SR_Flag',
 'Affiliated_base_number']

# 1. How many taxi trips were there on February 15?

## Cara Pertama

In [6]:
spark.sql("""
SELECT
    count(pickup_datetime) as count
FROM
    fhvhv_data
WHERE
    DATE(pickup_datetime) = '2021-02-15' 
""").show()

+-----+
|count|
+-----+
|35709|
+-----+



## Cara Kedua

In [7]:
spark.sql('''
select
    count(pickup_datetime) as count
from fhvhv_data
where
    pickup_datetime >= '2021-02-15' and pickup_datetime < '2021-02-16'
''').show()

+-----+
|count|
+-----+
|35709|
+-----+



## Cara Ketiga

In [8]:
spark.sql("""
SELECT 
    COUNT(pickup_datetime) as count
FROM 
    fhvhv_data
WHERE
    pickup_datetime BETWEEN '2021-02-15 00:00:00' AND '2021-02-15 23:59:59'
""").show()

+-----+
|count|
+-----+
|35709|
+-----+



# 2. Find the longest trip for each day ?

In [36]:
spark.sql("""
SELECT
    DATE(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60) AS duration
FROM 
    fhvhv_data
GROUP BY
    1
ORDER BY
    1 ASC
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2021-02-01|           46290.0|
| 2021-02-02|1390.7833333333333|
| 2021-02-03|1246.1666666666667|
| 2021-02-04| 40034.88333333333|
| 2021-02-05|          110919.0|
| 2021-02-06| 2752.633333333333|
| 2021-02-07|1306.1166666666666|
| 2021-02-08| 9424.916666666666|
| 2021-02-09|1459.9833333333333|
| 2021-02-10|1407.7166666666667|
| 2021-02-11|3219.8166666666666|
| 2021-02-12|            4344.0|
| 2021-02-13| 8422.683333333332|
| 2021-02-14|            1519.4|
| 2021-02-15|          14670.15|
| 2021-02-16|            4816.1|
| 2021-02-17| 4284.783333333334|
| 2021-02-18|2749.0333333333333|
| 2021-02-19|           9012.15|
| 2021-02-20|2701.4666666666667|
+-----------+------------------+
only showing top 20 rows



# 3. Find Top 5 Most frequent `dispatching_base_num` ?

In [10]:
spark.sql("""
SELECT
    dispatching_base_num, COUNT(dispatching_base_num)
FROM
    fhvhv_data
GROUP BY
    dispatching_base_num
ORDER BY
    COUNT(dispatching_base_num) DESC
LIMIT 5
""").show()

+--------------------+---------------------------+
|dispatching_base_num|count(dispatching_base_num)|
+--------------------+---------------------------+
|              B00856|                      35077|
|              B01312|                      33089|
|              B01145|                      31114|
|              B02794|                      30397|
|              B03016|                      29794|
+--------------------+---------------------------+



# 4. Find Top 5 Most common location pairs (PUlocationID and DOlocationID) ?

In [37]:
spark.sql("""
SELECT 
    CONCAT(PULocationID, '/', DOLocationID) AS pickup_dropoff_pair,
    COUNT(CONCAT(PULocationID, '/', DOLocationID)) AS count
FROM 
    fhvhv_data
GROUP BY
    CONCAT(PULocationID, '/', DOLocationID)
ORDER BY
    count DESC
LIMIT 5
""").show()

+-------------------+-----+
|pickup_dropoff_pair|count|
+-------------------+-----+
|        206.0/206.0| 2374|
|        221.0/206.0| 2112|
|        129.0/129.0| 1902|
|            7.0/7.0| 1829|
|        179.0/179.0| 1736|
+-------------------+-----+

